In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#plt.style.use("fivethirtyeight")
#%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
pretrained_mnist_model = tf.keras.models.load_model("/content/drive/MyDrive/pretrained_mnist_model.h5")

In [4]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [9]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable=False
  print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False


In [10]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name} : {layer.trainable}")

flatten : False
dense : False
leaky_re_lu : False
dense_1 : False
leaky_re_lu_1 : False
dense_2 : True


In [13]:
model_new=tf.keras.models.Sequential(pretrained_mnist_model.layers[:-1])
model_new.add(tf.keras.layers.Dense(2,activation='softmax'))

In [15]:
model_new.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 265,802
Trainable params: 202
Non-trainable params: 265,600
______________________________________________

In [16]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label >5, 1, 0)
  return labels

In [17]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [19]:
model_new.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [20]:
history = model_new.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 0.5487 - accuracy: 0.7228 - val_loss: 0.4617 - val_accuracy: 0.7882
Epoch 2/10
1719/1719 - 2s - loss: 0.4275 - accuracy: 0.8137 - val_loss: 0.4035 - val_accuracy: 0.8246
Epoch 3/10
1719/1719 - 2s - loss: 0.3914 - accuracy: 0.8363 - val_loss: 0.3753 - val_accuracy: 0.8416
Epoch 4/10
1719/1719 - 2s - loss: 0.3720 - accuracy: 0.8470 - val_loss: 0.3576 - val_accuracy: 0.8530
Epoch 5/10
1719/1719 - 3s - loss: 0.3595 - accuracy: 0.8527 - val_loss: 0.3460 - val_accuracy: 0.8592
Epoch 6/10
1719/1719 - 2s - loss: 0.3505 - accuracy: 0.8567 - val_loss: 0.3374 - val_accuracy: 0.8646
Epoch 7/10
1719/1719 - 2s - loss: 0.3436 - accuracy: 0.8609 - val_loss: 0.3305 - val_accuracy: 0.8666
Epoch 8/10
1719/1719 - 2s - loss: 0.3379 - accuracy: 0.8633 - val_loss: 0.3244 - val_accuracy: 0.8702
Epoch 9/10
1719/1719 - 3s - loss: 0.3332 - accuracy: 0.8654 - val_loss: 0.3196 - val_accuracy: 0.8714
Epoch 10/10
1719/1719 - 2s - loss: 0.3292 - accuracy: 0.8670 - val_loss: 0.3163 - 

In [21]:
model_new.save("/content/drive/MyDrive/mnist_model_gt5.h5")

In [23]:
model_new.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3201 - accuracy: 0.8715


[0.3200502097606659, 0.8715000152587891]

In [24]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([1, 0, 0]))

In [26]:
np.argmax(model_new.predict(X_new), axis=-1)

array([1, 0, 0])